<a href="https://colab.research.google.com/github/HollowMike8/bounding-box-regression-keras/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/HollowMike8/bounding-box-regression-keras.git

In [ ]:
import os
import cv2
import mimetypes
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
path_dir: str = r"/content/bounding-box-regression-keras"

import sys
sys.path.insert(0,path_dir)
from pyimagesearch import config

In [ ]:
input = os.path.join(config.test_filenames, "test_images.txt")

# check if the input is a single image or a list of images in txt file
filetype = mimetypes.guess_type(input)[0]

if "text/plain" == filetype:
  filenames = open(input, "r").read().strip().split("\n")
  imagePaths = []

  for filename in filenames:
    p = os.path.join(config.images_path, filename)
    imagePaths.append(p)
elif "image/jpeg" == filetype or "image/png" == filetype:
  imagePaths = [input]

In [ ]:
# load the trained bounding box regressor
model = load_model(config.model_path)

# perform a forward pass on the test image/images
for i, imagePath in enumerate(imagePaths):
  image = load_img(imagePath, target_size = (224, 224))
  image = img_to_array(image)/255.0
  image = np.expand_dims(image, axis=0)

  preds = model.predict(image)[0]
  (startX, startY, endX, endY) = preds

  # read the test image an upscale the bounding box coordinates
  image = cv2.imread(imagePath)
  (h,w) = image.shape[:2]

  startX = int(startX*w)
  startY = int(startY*h)
  endX = int(endX*w)
  endY = int(endY*h)

  # draw the bounding box over the object
  cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)

  # plot the image with bounding box
  plt.imshow(image)
  plt.axis("off")
  plt.title("Output for %s"% filenames[i])
  plt.show()